In [157]:
import pandas as pd
from datetime import datetime, timedelta
from datetime import date
from pandas import date_range
from workalendar.asia import SouthKorea
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from datetime import datetime, date, timedelta
from tqdm.notebook import tqdm 

# 크롤링(2022~2023)데이터 전처리 및  공공데이터 (2012~2022) 병합과정


<b>*Data Input*</b>
- 2012~2022전력수급.csv
- craw_fin_2022_2023.csv

In [158]:
#크롤링 결과 데이터 
df = pd.read_csv("./Data_Raw/craw_fin_2022_2023.csv")

In [159]:
merged_df1 = df.copy()
merged_df1 = merged_df1.drop(merged_df1.columns[0], axis=1)
merged_df1 = merged_df1.sort_values('기준시간')
merged_df1 = merged_df1.reset_index(drop=True)
merged_df1 = merged_df1[:-1]
merged_df1['기준시간'] = pd.to_datetime(merged_df1['기준시간'], format='%Y-%m-%d %H:%M:%S')
md2 = merged_df1.copy()
columns_to_process = ['공급능력', '현재부하', '공급예비력', '운영예비력']
for column in columns_to_process:
    md2[column] = md2[column].str.replace(',', '').astype(float)

md2['공급능력'] = md2['공급능력'].astype(int)
md2['현재부하'] = md2['현재부하'].astype(int)
md2['공급예비력'] = md2['공급예비력'].astype(int)
md2['운영예비력'] = md2['운영예비력'].astype(int)
md2 = md2.rename(columns={
    '기준시간': '기준일시', 
    '공급능력': '공급능력(MW)',
    '현재부하': '현재수요(MW)', 
    '공급예비력': '공급예비력(MW)',
    '운영예비력': '운영예비력(MW)',
    '운영예비율': '운영예비율(퍼센트)'
})


In [160]:
md2 = md2.drop(columns=['운영예비율(퍼센트)'])
md2['운영예비율(퍼센트)'] = (md2['운영예비력(MW)'] / md2['현재수요(MW)'] )*100
#공급 예비율 만들기 
md2['공급예비율(퍼센트)'] = (md2['공급예비력(MW)'] / md2['공급능력(MW)']) * 100
md2['공급예비율(퍼센트)'] = ((md2['공급예비력(MW)'] / md2['공급능력(MW)']) * 100)


In [161]:
#단위 변환하기 
cols_to_update = ['공급능력(MW)', '현재수요(MW)', '공급예비력(MW)', '운영예비력(MW)']

# 각 열을 순회하면서 1000으로 나눔
for col in cols_to_update:
    md2[col] = md2[col] / 1000


In [162]:
#공공데이터포털의 2012.06~ 2022 04 까지의 데이터 
df2 = pd.read_csv('2012~2022전력수급.csv',encoding='cp949')

In [163]:
df2 = df2.drop('최대예측수요(MW)', axis=1)

df2

,기준일시,공급능력(MW),현재수요(MW),공급예비력(MW),공급예비율(퍼센트),운영예비력(MW),운영예비율(퍼센트)
0,2012-06-01 00:00,64355.00000,51726.33000,12628.67000,24.41439,12628.67000,24.41439
1,2012-06-01 00:05,64355.00000,51342.53000,13012.47000,25.34442,13012.47000,25.34442
2,2012-06-01 00:10,64355.00000,51583.17000,12771.83000,24.75969,12771.83000,24.75969
3,2012-06-01 00:15,64355.00000,51631.69000,12723.31000,24.64245,12723.31000,24.64245
4,2012-06-01 00:20,64355.00000,51252.66000,13102.35000,25.56423,13102.35000,25.56423
...,...,...,...,...,...,...,...
1035062,2022-04-07 23:35,82791.90000,63877.30000,18914.70000,29.61100,10803.00000,16.91210
1035063,2022-04-07 23:40,82768.10000,63525.30000,19242.80000,30.29160,10696.40000,16.83800
1035064,2022-04-07 23:45,82767.90000,63506.20000,19261.70000,30.33040,10501.30000,16.53580
1035065,2022-04-07 23:50,82762.40000,62819.60000,19942.80000,31.74610,10706.30000,17.04290


In [165]:
md2['기준일시'] = pd.to_datetime(md2['기준일시'])

# 이후의 과정은 이전과 동일
md2 = md2[md2['기준일시'] >= '2022-04-08']
df = pd.concat([df2, md2], ignore_index=True)
df

,기준일시,공급능력(MW),현재수요(MW),공급예비력(MW),공급예비율(퍼센트),운영예비력(MW),운영예비율(퍼센트)
0,2012-06-01 00:00,64355.00000,51726.33000,12628.67000,24.41439,12628.67000,24.41439
1,2012-06-01 00:05,64355.00000,51342.53000,13012.47000,25.34442,13012.47000,25.34442
2,2012-06-01 00:10,64355.00000,51583.17000,12771.83000,24.75969,12771.83000,24.75969
3,2012-06-01 00:15,64355.00000,51631.69000,12723.31000,24.64245,12723.31000,24.64245
4,2012-06-01 00:20,64355.00000,51252.66000,13102.35000,25.56423,13102.35000,25.56423
...,...,...,...,...,...,...,...
1135286,2023-03-22 23:35:00,86286.00000,63432.00000,22436.30000,26.00225,11623.30000,18.32403
1135287,2023-03-22 23:40:00,86287.90000,62935.10000,23087.90000,26.75682,11587.50000,18.41182
1135288,2023-03-22 23:45:00,86286.50000,62616.60000,23670.00000,27.43187,11484.20000,18.34050
1135289,2023-03-22 23:50:00,86274.60000,62236.50000,24038.00000,27.86220,11538.50000,18.53976


# 병합데이터 전처리과정

In [166]:
df['기준일시'] = pd.to_datetime(df['기준일시'])
df['요일'] = df['기준일시'].dt.dayofweek

# Start and end dates
start_year = 2012
end_year = 2023

cal = SouthKorea()

holidays = [day[0] for year in range(start_year, end_year + 1) for day in cal.holidays(year)]

weekends_sat = [d.date() for d in date_range(start=f'{start_year}-06-01', end=f'{end_year}-12-31', freq='W-SAT')]
weekends_sun = [d.date() for d in date_range(start=f'{start_year}-06-01', end=f'{end_year}-12-31', freq='W-SUN')]


all_days_off = holidays + weekends_sat + weekends_sun
all_days_off = list(set(all_days_off))
all_days_off.sort()

print(all_days_off)

[datetime.date(2012, 1, 1), datetime.date(2012, 1, 22), datetime.date(2012, 1, 23), datetime.date(2012, 1, 24), datetime.date(2012, 3, 1), datetime.date(2012, 4, 28), datetime.date(2012, 5, 5), datetime.date(2012, 6, 2), datetime.date(2012, 6, 3), datetime.date(2012, 6, 6), datetime.date(2012, 6, 9), datetime.date(2012, 6, 10), datetime.date(2012, 6, 16), datetime.date(2012, 6, 17), datetime.date(2012, 6, 23), datetime.date(2012, 6, 24), datetime.date(2012, 6, 30), datetime.date(2012, 7, 1), datetime.date(2012, 7, 7), datetime.date(2012, 7, 8), datetime.date(2012, 7, 14), datetime.date(2012, 7, 15), datetime.date(2012, 7, 21), datetime.date(2012, 7, 22), datetime.date(2012, 7, 28), datetime.date(2012, 7, 29), datetime.date(2012, 8, 4), datetime.date(2012, 8, 5), datetime.date(2012, 8, 11), datetime.date(2012, 8, 12), datetime.date(2012, 8, 15), datetime.date(2012, 8, 18), datetime.date(2012, 8, 19), datetime.date(2012, 8, 25), datetime.date(2012, 8, 26), datetime.date(2012, 9, 1), date

In [ ]:
df['date'] = df['기준일시'].dt.date
df['is_holiday_or_weekend'] = df['date'].apply(lambda x: 1 if x in all_days_off else 0)
df = df.drop(columns=['date'])

df.head()

,기준일시,공급능력(MW),현재수요(MW),공급예비력(MW),공급예비율(퍼센트),운영예비력(MW),운영예비율(퍼센트),요일,is_holiday_or_weekend
0,2012-06-01 00:00:00,64355.00000,51726.33000,12628.67000,24.41439,12628.67000,24.41439,4,0
1,2012-06-01 00:05:00,64355.00000,51342.53000,13012.47000,25.34442,13012.47000,25.34442,4,0
2,2012-06-01 00:10:00,64355.00000,51583.17000,12771.83000,24.75969,12771.83000,24.75969,4,0
3,2012-06-01 00:15:00,64355.00000,51631.69000,12723.31000,24.64245,12723.31000,24.64245,4,0
4,2012-06-01 00:20:00,64355.00000,51252.66000,13102.35000,25.56423,13102.35000,25.56423,4,0


In [167]:
#요일 벡터화 과정 
def convert_to_binary(day):
    binary_mapping = {
        0: [0, 0, 1],  # 월요일
        1: [0, 1, 0],  # 화요일
        2: [0, 1, 1],  # 수요일
        3: [1, 0, 0],  # 목요일
        4: [1, 0, 1],  # 금요일
        5: [1, 1, 0],  # 토요일
        6: [1, 1, 1]   # 일요일
    }
    
    return binary_mapping[day]

df[['요일1', '요일2', '요일3']] = df['요일'].apply(convert_to_binary).tolist()

In [168]:
df

,기준일시,공급능력(MW),현재수요(MW),공급예비력(MW),공급예비율(퍼센트),운영예비력(MW),운영예비율(퍼센트),요일,요일1,요일2,요일3
0,2012-06-01 00:00:00,64355.00000,51726.33000,12628.67000,24.41439,12628.67000,24.41439,4,1,0,1
1,2012-06-01 00:05:00,64355.00000,51342.53000,13012.47000,25.34442,13012.47000,25.34442,4,1,0,1
2,2012-06-01 00:10:00,64355.00000,51583.17000,12771.83000,24.75969,12771.83000,24.75969,4,1,0,1
3,2012-06-01 00:15:00,64355.00000,51631.69000,12723.31000,24.64245,12723.31000,24.64245,4,1,0,1
4,2012-06-01 00:20:00,64355.00000,51252.66000,13102.35000,25.56423,13102.35000,25.56423,4,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...
1135286,2023-03-22 23:35:00,86286.00000,63432.00000,22436.30000,26.00225,11623.30000,18.32403,2,0,1,1
1135287,2023-03-22 23:40:00,86287.90000,62935.10000,23087.90000,26.75682,11587.50000,18.41182,2,0,1,1
1135288,2023-03-22 23:45:00,86286.50000,62616.60000,23670.00000,27.43187,11484.20000,18.34050,2,0,1,1
1135289,2023-03-22 23:50:00,86274.60000,62236.50000,24038.00000,27.86220,11538.50000,18.53976,2,0,1,1


In [ ]:
# 해당 과정까지 결과를 저장
#df.to_csv('6_20.csv',index = False)

# 기상데이터(weather_data_v1.csv) 전처리 및 병합과정  
*Input Data*
 - 6_20.csv(앞선 데이터 병합의 결과)
 - weather_data_v1.csv(기상 정보 로우데이터)

*최종 결과물*
 - weather_electric_data.csv

In [172]:
# 상기 결과를 카피
adding_df = df.copy()

In [ ]:
df = pd.read_csv('./data/weather_data_v1.csv')

df['일시'] = pd.to_datetime(df['일시'])

# 지점, 일시 순으로 정렬
df.sort_values(by=['지점', '일시'], inplace=True, ignore_index=True)

# 누락 시간대 데이터프레임
adding_df = pd.DataFrame(columns=df.columns)

# 누락 시간대에 행 추가 - 앞뒤 평균치로 대체
for i in tqdm(range(len(df)-1)):

    diff = df['일시'][i+1] - df['일시'][i]
    
    # 앞뒤 행의 차이가 5분이 아닐 경우
    if diff.seconds != 300:

        if df['지점'][i+1] == df['지점'][i]:
            print(df['지점명'][i], df['일시'][i], diff)

            for j in range(int((diff.seconds) / 300 - 1)):
                # 새 행 추가
                new_row = {
                    '지점' : df['지점'][i],
                    '지점명': df['지점명'][i],
                    '일시' : df['일시'][i] + timedelta(minutes=5*(j+1)),
                    '기온(°C)' : (df['기온(°C)'][i] + df['기온(°C)'][i+1]) / 2,
                    '누적강수량(mm)' : (df['누적강수량(mm)'][i] + df['누적강수량(mm)'][i+1]) / 2,
                    '풍향(deg)' : (df['풍향(deg)'][i] + df['풍향(deg)'][i+1]) / 2,
                    '풍속(m/s)' : (df['풍속(m/s)'][i] + df['풍속(m/s)'][i+1]) / 2,
                    '현지기압(hPa)' : (df['현지기압(hPa)'][i] + df['현지기압(hPa)'][i+1]) / 2,
                    '해면기압(hPa)' : (df['해면기압(hPa)'][i] + df['해면기압(hPa)'][i+1]) / 2,
                    '습도(%)' : (df['습도(%)'][i] + df['습도(%)'][i+1]) / 2,
                    '일사(MJ/m^2)' : (df['일사(MJ/m^2)'][i] + df['일사(MJ/m^2)'][i+1]) / 2,
                    '일조(Sec)' : (df['일조(Sec)'][i] + df['일조(Sec)'][i+1]) / 2
                }

                adding_df = adding_df.append(new_row, ignore_index=True)


df = pd.concat([df,adding_df], ignore_index=True)
df['일시'] = pd.to_datetime(df['일시'])
df.sort_values(by=['지점', '일시'], inplace=True, ignore_index=True)


# csv 저장
# df.to_csv('weather_data_v3.csv', index=False)


# 시간대별 데이터 만들기
df = pd.read_csv('weather_data_v3.csv')

df.sort_values(by=['일시', '지점'], inplace=True, ignore_index=True)

# 새 데이터프레임 만들기
new_cols = ['일시']
for city in df['지점명'].unique():
    for info in df.columns[3:]:
        new_cols.append(f'{city}_{info}')
        
new_df = pd.DataFrame(columns=new_cols, index=range(1135872))
new_df['일시'] = df['일시'].unique()

count = 0
# 같은 시간대 - 16,32,48,..
for i in tqdm(range(16,len(df),16)):
    # 한 지역 - 0,1,2,...
    col_set = 1
    for j in range(i-16, i):
        new_df.loc[count][col_set:col_set+9] = df.loc[j][3:]
        col_set += 9
    count += 1
    
new_df = new_df.astype('float')

# csv 저장
new_df.to_csv('weather_data_v4.csv', index=False)

new_df = pd.read_csv('weather_data_v4.csv')

new_df.set_index('일시', drop=True, inplace=True)
new_df.fillna(new_df.interpolate(), inplace=True)


# 전력 데이터 불러오기
electric1 = pd.read_csv('6_20.csv')

electric1['기준일시'] = pd.to_datetime(electric1['기준일시'])

adding_df = pd.DataFrame(columns=electric1.columns)

# 누락 시간대에 행 추가 - 평균치로 대체
for i in tqdm(range(len(electric1)-1)):

    diff = electric1['기준일시'][i+1] - electric1['기준일시'][i]
    
    # 앞뒤 행의 차이가 5분이 아닐 경우
    if (diff.days != 0) or (diff.seconds != 300):

        print(electric1['기준일시'][i], diff)

        for j in range(int((diff.days*86400 + diff.seconds) / 300 - 1)):
            # 새 행 추가
            new_row = {
                '기준일시' : electric1['기준일시'][i] + timedelta(minutes=5*(j+1)),
                '공급능력(MW)' : (electric1['공급능력(MW)'][i] + electric1['공급능력(MW)'][i+1]) / 2,
                '현재수요(MW)' : (electric1['현재수요(MW)'][i] + electric1['현재수요(MW)'][i+1]) / 2,
                '공급예비력(MW)' : (electric1['공급예비력(MW)'][i] + electric1['공급예비력(MW)'][i+1]) / 2,
                '공급예비율(퍼센트)' : (electric1['공급예비율(퍼센트)'][i] + electric1['공급예비율(퍼센트)'][i+1]) / 2 ,               
                '운영예비력(MW)' : (electric1['운영예비력(MW)'][i] + electric1['운영예비력(MW)'][i+1]) / 2,
                '운영예비율(퍼센트)' : (electric1['운영예비율(퍼센트)'][i] + electric1['운영예비율(퍼센트)'][i+1]) / 2,
                '요일' : electric1['요일'][i],
                'is_holiday_or_weekend' : electric1['is_holiday_or_weekend'][i],
                '요일1' : electric1['요일1'][i],
                '요일2' : electric1['요일2'][i],
                '요일3' : electric1['요일3'][i],
            }
            adding_df = adding_df.append(new_row, ignore_index=True)


electric1 = pd.concat([electric1,adding_df], ignore_index=True)
electric1['기준일시'] = pd.to_datetime(electric1['기준일시'])
electric1.sort_values(by=['기준일시'], inplace=True, ignore_index=True)

# 기상 데이터 + 전력 데이터
df = new_df.join(electric1)

df.to_csv('weather_electric_data.csv')

# weather_electirc_data 이후  EDA 전처리

*Input Data*
 - weather_electric_data.csv(앞선 데이터 처리의 결과)

*최종 결과물*
 - weather_electric_data.csv

In [176]:
df = pd.read_csv('./data/weather_electric_data.csv')

In [177]:
df['date'] = pd.to_datetime(df['date'])
df.set_index('date', drop=True, inplace=True)

- 현재수요 0인데이터처리 이전행대체 

In [181]:
error_date = df[df['현재수요(MW)'] == 0].index
error_date_length = len(error_date)

for i in range(error_date_length):
    df.loc[error_date[i], '현재수요(MW)'] = df.loc[error_date[i] - timedelta(minutes=5), '현재수요(MW)']
# error_date = df[df['현재수요(MW)'] == 0].index
# for i in tqdm(range(len(error_date))):
#     df.loc[error_date[i], '현재수요(MW)'] = df.loc[error_date[i] - timedelta(minutes=5), '현재수요(MW)']

- 여수 : 170322 1개 행, 서울 : 171105 다수 행에서 문제
- 재수집 결과, 기상청 데이터 자체에서 문제
    - 여수는 앞뒤 정상적인 온도로 대체
    - 서울은 전날 같은 시간의 온도로 대체

In [182]:
df.loc['2017-03-22 13:35:00', '여수_기온(°C)'] = np.mean([df.loc['2017-03-22 13:30:00', '여수_기온(°C)'], df.loc['2017-03-22 13:40:00', '여수_기온(°C)']])
df.loc['2017-11-05 13:00:00' : '2017-11-06 00:00:00', '서울_기온(°C)'] = df.loc['2017-11-04 13:00:00' : '2017-11-05 00:00:00', '서울_기온(°C)'].values

- 여수 풍향 조절 

170322 모두 전날 (170321) 데이터 사용

In [183]:
df.loc[df['여수_풍향(deg)'] < 0, '여수_풍향(deg)']

date
2017-03-22 11:20:00    -3.56786
2017-03-22 11:25:00    -7.13571
2017-03-22 11:30:00   -10.70357
2017-03-22 11:35:00   -14.27143
2017-03-22 11:40:00   -17.83929
                         ...   
2017-03-22 18:55:00    -7.23913
2017-03-22 19:00:00    -5.79130
2017-03-22 19:05:00    -4.34348
2017-03-22 19:10:00    -2.89565
2017-03-22 19:15:00    -1.44783
Name: 여수_풍향(deg), Length: 96, dtype: float64

In [184]:
df.loc['2017-03-22', '여수_풍향(deg)'] = df.loc['2017-03-21', '여수_풍향(deg)'].values

- 서울, 여수 처리 필요
- 서울은 앞 행으로 대체
- 여수는 전날로 대체

In [185]:
df.loc['2017-10-14 18:15:00':'2017-10-14 18:20:00', '서울_풍속(m/s)'] = np.mean([df.loc['2017-10-14 18:10:00', '서울_풍속(m/s)'], df.loc['2017-10-14 18:25:00','서울_풍속(m/s)']])
df.loc['2017-03-22', '여수_풍속(m/s)'] = df.loc['2017-03-21', '여수_풍속(m/s)'].values

- 현지기압


In [186]:
df.loc['2017-03-22', '여수_현지기압(hPa)'] = df.loc['2017-03-21', '여수_현지기압(hPa)'].values

In [187]:
df.loc[df['부산_현지기압(hPa)'] < 950, '부산_현지기압(hPa)']

date
2020-03-10 10:35:00   828.50000
2022-11-17 11:00:00   765.70000
Name: 부산_현지기압(hPa), dtype: float64

- 5분전으로 대체

In [188]:
df.loc['2020-03-10 10:35:00', '부산_현지기압(hPa)'] = df.loc['2020-03-10 10:30:00', '부산_현지기압(hPa)']
df.loc['2022-11-17 11:00:00', '부산_현지기압(hPa)'] = df.loc['2022-11-17 10:55:00', '부산_현지기압(hPa)']

- 해면기압   
서울 여수 처리  
대전 부산 확인 

In [189]:
df.loc['2017-03-22', '여수_해면기압(hPa)'] = df.loc['2017-03-21', '여수_해면기압(hPa)'].values

In [190]:
df.loc['2017-11-05', '서울_해면기압(hPa)'] = df.loc['2017-11-04', '서울_해면기압(hPa)'].values

In [191]:
df.loc['2020-03-10 10:35:00', '부산_해면기압(hPa)'] = df.loc['2020-03-10 10:30:00', '부산_해면기압(hPa)']
df.loc['2022-11-17 11:00:00', '부산_해면기압(hPa)'] = df.loc['2022-11-17 10:55:00', '부산_해면기압(hPa)']
df.loc['2021-12-24 10:35:00', '대전_해면기압(hPa)'] = df.loc['2021-12-24 10:30:00', '대전_해면기압(hPa)']

- 여수 습도처리

In [192]:
df.loc[df['여수_습도(%)'] < 0, '여수_습도(%)']

date
2017-03-22 12:00:00    -2.79643
2017-03-22 12:05:00    -7.90714
2017-03-22 12:10:00   -13.01786
2017-03-22 12:15:00   -18.12857
2017-03-22 12:20:00   -23.23929
                         ...   
2017-03-22 19:55:00    -5.80476
2017-03-22 20:00:00    -4.56667
2017-03-22 20:05:00    -3.32857
2017-03-22 20:10:00    -2.09048
2017-03-22 20:15:00    -0.85238
Name: 여수_습도(%), Length: 100, dtype: float64

- 일사   
- 울산과 천안의 경우, 일사 데이터가 대체된 경우가 많으므로, 위치가 비슷한 지역으로 대체
    - 울산 -> 포항
    - 천안 -> 청주
  
- 여수 처리 필요

- 일조 처리 서울 처리 필요

In [193]:
df.loc[df['서울_일조(Sec)'] < 0, '서울_일조(Sec)']

date
2017-11-05 13:00:00   -999.00000
2017-11-05 13:05:00   -991.48872
2017-11-05 13:10:00   -983.97744
2017-11-05 13:15:00   -976.46617
2017-11-05 13:20:00   -968.95489
                         ...    
2017-11-05 23:40:00    -37.55639
2017-11-05 23:45:00    -30.04511
2017-11-05 23:50:00    -22.53383
2017-11-05 23:55:00    -15.02256
2017-11-06 00:00:00     -7.51128
Name: 서울_일조(Sec), Length: 133, dtype: float64

In [194]:
df.loc['2017-11-05', '서울_일조(Sec)'] = df.loc['2017-11-04', '서울_일조(Sec)'].values

- 해면기압 일사 제거

In [195]:
df.drop(df.columns[df.columns.str.contains('해면기압') | df.columns.str.contains('일사')], axis=1, inplace=True)

In [ ]:
# 해당 과정까지 결과를 저장
# df.to_csv('After_eda.csv',index = False)

# 전처리 이후 가중평균 열로 통합 및 데이터 필터준비 (모델전)

*Input Data*
 - After_eda.csv(앞선 데이터 처리의 결과)
 - 지역별 주거용 전력판매량.csv(지역별 전력판매량을 바탕으로, 기상정보 가중치를 산정함에 사용함)

*최종 결과물*
 - Final_Preprocessed_Data.csv (날짜 인덱싱을 하지 않은 데이터 약 11년치)

In [198]:
df_1 = df.copy().reset_index(inplace=True)

df_1 = df_1.drop(columns=['요일'])
df_1.to_csv('7_9.csv',index=False)

In [202]:
from util2.Date_Processor import WeightedAverageCalculator

calculator = WeightedAverageCalculator('./data/지역별 주거용 전력판매량.csv', '7_9.csv', '연도')
result_df = calculator.calculate_weighted_average()

columns_to_keep = ['date', '현재수요(MW)'] + [col for col in result_df.columns if '가중평균' in col]
columns_to_drop = [col for col in result_df.columns if col not in columns_to_keep]
result_df.drop(columns_to_drop, axis=1, inplace=True)

In [203]:
result_df

,date,현재수요(MW),전체지역_기온(°C)_가중평균,전체지역_누적강수량(mm)_가중평균,전체지역_풍향(deg)_가중평균,전체지역_풍속(m/s)_가중평균,전체지역_현지기압(hPa)_가중평균,전체지역_습도(%)_가중평균,전체지역_일조(Sec)_가중평균
0,2012-06-01 00:05:00,51342.53000,17.20815,0.00000,94.62696,0.96372,1010.20467,81.30841,155.34185
1,2012-06-01 00:10:00,51583.17000,17.08440,0.00000,85.15243,0.83922,1010.13500,81.81100,155.34185
2,2012-06-01 00:15:00,51631.69000,17.03752,0.00000,71.42581,0.82191,1010.05435,82.18481,155.34185
3,2012-06-01 00:20:00,51252.66000,17.00843,0.00000,70.96398,0.85057,1009.95739,82.28891,155.34185
4,2012-06-01 00:25:00,50888.38000,16.93998,0.00000,72.56493,0.75381,1009.96015,82.48213,155.34185
...,...,...,...,...,...,...,...,...,...
1135866,2023-03-19 23:35:00,58627.20000,5.22279,0.00000,223.45016,1.57493,1011.33646,73.95091,39452.92427
1135867,2023-03-19 23:40:00,57941.20000,5.14996,0.00000,225.98202,1.65918,1011.28536,74.20099,39452.92427
1135868,2023-03-19 23:45:00,57726.20000,5.07690,0.00000,228.85142,1.51663,1011.28012,74.18378,39452.92427
1135869,2023-03-19 23:50:00,57755.40000,5.03747,0.00000,221.91882,1.45383,1011.30961,74.25380,39452.92427
